In [259]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import itertools

pd.options.mode.chained_assignment = None  # default='warn'

In [260]:
Racesdf = pd.read_csv('Datasets/races.csv', sep=',')
Racesdf.sort_values(by="date", inplace=True)
Racesdf["race"] = Racesdf["year"].astype(str) + " " + Racesdf["name"]
Racesdf = Racesdf[["raceId", "year", "round", "race"]]

Driversdf = pd.read_csv('Datasets/drivers.csv', sep=',')
Driversdf["name"] =Driversdf["forename"] + " " + Driversdf["surname"]
Driversdf = Driversdf[["driverId", "name"]]


Resultsdf = pd.read_csv('Datasets/results.csv', sep=',')
Resultsdf = Resultsdf[["resultId", "raceId", "driverId", "position", "positionOrder"]]
Resultsdf= Resultsdf.replace(r"\\N", "DNF" , regex=True)


In [261]:
InitialElo = 1300
kValue = 9
Drivers = {}
Races ={}

In [262]:
def RacePositions(RaceID, include = "all"):
    if include == "all":
        Positions = Resultsdf.loc[Resultsdf["raceId"]== RaceID, ["positionOrder", "driverId", "raceId"]]
        Positions["position"] = Positions["positionOrder"]
    elif include == "finished":
        Positions = Resultsdf.loc[Resultsdf["raceId"]== RaceID, ["position", "driverId", "raceId"]]
        Positions = Positions.loc[Positions["position"] != "DNF"]
        Positions["position"] = Positions["position"].astype(int)
    else:
        raise ValueError("include= all or finished")
    
    Positions.sort_values(by="position", inplace=True)
    Positions['nDrivers'] = Positions.groupby('position')['position'].transform('count')
    return Positions[["position", "driverId", "nDrivers"]]

def EloChange(Elo1, Elo2, Result="Win", nDrivers1=1, nDrivers2=1):
    K = kValue/nDrivers1/nDrivers2
    P1 = (1/(1+10**((Elo2-Elo1)/400)))
    P2 = (1/(1+10**((Elo1-Elo2)/400)))
    if Result == "Win":
        NewElo1 = K*(1.0 - P1)
        NewElo2 = K*(0.0 - P2)
    elif Result == "Draw":
        NewElo1 = K*(0.5 - P1)
        NewElo2 = K*(0.5 - P2)
    elif Result == "Loss":
        NewElo1 = K*(0.0 - P1)
        NewElo2 = K*(1.0 - P2)
    else:
        raise ValueError("EloChange result must be Win, Draw or Loss")
    return NewElo1, NewElo2


In [273]:
def RaceCalculator(RaceId, positions):
    raceName = list(Racesdf.loc[Racesdf["raceId"]==RaceId, "race"])[0]
    Changes = {}
    Races[RaceName] = {}




    
    Races[Race][driver] = Drivers[driver]["Elo"]
    return raceName

def RaceCalculator(result):
    Race = result["Race"].values[0]
    ResultDict = dict(zip(result.Driver, result.position))
    Changes = {}
    Races[Race] = {}
    
    for driver in ResultDict.keys():
        if driver not in Drivers:
            AddDriver(driver)
            Drivers[driver]["First Race"] = Race
        Changes[driver] = 0
        Drivers[driver]["Last Race"] = Race
     
    for driver1, driver2 in itertools.combinations(ResultDict.keys(), 2):
        if ResultDict[driver1] < ResultDict[driver2]:
            a = "Win"
        elif ResultDict[driver1] == ResultDict[driver2]:
            a = "Draw"
        elif ResultDict[driver1] > ResultDict[driver2]:
            a = "Loss"
        else: 
            raise ValueError("Something Happened")
        change1, change2 = EloChange(Drivers[driver1]["Elo"], Drivers[driver2]["Elo"], a)
        Changes[driver1] += change1
        Changes[driver2] += change2

    for driver in ResultDict.keys():
        if ResultDict[driver] == 1:
            Drivers[driver]["Wins"] += 1
        Drivers[driver]["Elo"] += Changes[driver]
        Drivers[driver]["Entries"] += 1
        if Drivers[driver]["Elo"] >= Drivers[driver]["Max Elo"]:
            Drivers[driver]["Max Elo"] = Drivers[driver]["Elo"]
            Drivers[driver]["Max Elo Race"] = Race
        if Drivers[driver]["Elo"] <= Drivers[driver]["Min Elo"]:
            Drivers[driver]["Min Elo"] = Drivers[driver]["Elo"]
            Drivers[driver]["Min Elo Race"] = Race
        Races[Race][driver] = Drivers[driver]["Elo"]
    return


In [264]:
b= RacePositions(833, include="finished")
b

,position,driverId,nDrivers
20024,1,642,1
20025,2,786,1
20026,3,686,1
20027,4,704,1
20028,5,627,1
20029,6,619,1
20030,7,787,1
20031,8,741,1
20032,9,784,1
20033,10,778,2


In [272]:
RaceCalculator(833, b)

('1950 British Grand Prix',
 {642: 1,
  786: 2,
  686: 3,
  704: 4,
  627: 5,
  619: 6,
  787: 7,
  741: 8,
  784: 9,
  778: 10,
  788: 10,
  660: 11})